In [86]:
from datasets import load_dataset

sst2 = load_dataset("sst")
print(sst2["train"])

No config specified, defaulting to: sst/default
Reusing dataset sst (/Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 206.07it/s]

Dataset({
    features: ['sentence', 'label', 'tokens', 'tree'],
    num_rows: 8544
})


In [1]:
%load_ext autoreload  
%autoreload 2 

from subset_active_learning.active_learner import ActiveLearner, ActiveLearnerConfig
import wandb
%env WANDB_PROJECT=subset_active_learning_corrected
%env WANDB_NOTEBOOK_NAME="./train.ipynb"
wandb.login()

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find "./train.ipynb".
wandb: Currently logged in as: jjoozzz. Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=subset_active_learning_corrected
env: WANDB_NOTEBOOK_NAME="./train.ipynb"


True

In [54]:
from transformers import set_seed
set_seed(42)

In [80]:
pool_indices = 10
sampling_sizes = (1, 2, 3, 4)

all_samples = np.random.choice(pool_indices, replace=False, size=4) 

In [81]:
all_samples

array([1, 5, 2, 4])

In [83]:
sample1 = all_samples[:1]; sample1

array([1])

In [85]:
sample4 = all_samples[:4]; sample4

array([1, 5, 2, 4])

In [79]:
samples

[array([4, 8]), array([2, 0]), array([4, 2]), array([5, 2])]

In [74]:
sampled_data()

array([7, 5])

In [29]:
import pickle

train_datasets = []
for training_size in (1000, 2000, 3000, 4000):
    with open(f"./debug/train_ds_random_sampling_{training_size}.pkl", "rb") as f:
        train_ds = pickle.load(f)
        train_datasets.append(train_ds)

In [30]:
train_datasets[0]["input_ids"].shape

torch.Size([1000, 66])

In [31]:
train_datasets[3]["input_ids"].shape

torch.Size([4000, 66])

In [52]:
import torch

torch.eq(train_datasets[3]["input_ids"][0:1000], train_datasets[3]["input_ids"][1000:2000]).all()

tensor(True)

In [53]:
torch.eq(train_datasets[3]["input_ids"][2000:3000], train_datasets[3]["input_ids"][1000:2000]).all()

tensor(True)

In [2]:
config = ActiveLearnerConfig(debug=False, sampling_sizes=(1000, 2000, 3000, 4000), strategy="random_sampling")
config

ActiveLearnerConfig(max_length=66, debug=False, model_name='google/electra-small-discriminator', strategy='random_sampling', sampling_sizes=(1000, 2000, 3000, 4000), max_steps=20000, batch_size=8)

In [3]:
active_learner = ActiveLearner(config)

No config specified, defaulting to: sst/default
Reusing dataset sst (/Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 370.41it/s]
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-c6c7962a7d90e9c5.arrow
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-915eb92e28e9eeaf.arrow
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-90df394ccb2006db.arrow
Loading cached processed dataset at /Users/garylai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-b196b58ce5e6a588

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed

self = active_learner
n_new_samples = 1000

set_seed(42)

all_sampled_data = []
for i in range(4):
        sampled_data = self.sample_data(n_new_samples)
        all_sampled_data.append(sampled_data)
        # concatenate the sampled data with the original data
# self.train_data_indices.extend(sampled_data)
# train_data = self.sst2["train"].select(self.train_data_indices)

# training_args = TrainingArguments(
#             output_dir="./dir",
#             max_steps=self.config.max_steps if not self.config.debug else 640,
#             evaluation_strategy="steps",
#             report_to="wandb",
#             run_name=f"{self.config.strategy}-size-{len(self.train_data_indices)}",
#             eval_steps=300,
#             learning_rate=1e-5,
#             adam_epsilon=1e-6,
#             warmup_ratio=0.1,
#             weight_decay=0.01,
#         )

# train_ds = self.preprocess(train_data)

# print(f"training_args: {training_args}")
# model = AutoModelForSequenceClassification.from_pretrained(self.config.model_name, num_labels=2)
# trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_ds if not self.config.debug else self.debug_ds,
#         compute_metrics=self.compute_metrics,
# )

config strategy is random_sampling
config strategy is random_sampling
config strategy is random_sampling
config strategy is random_sampling


In [19]:
import numpy as np

np.testing.assert_array_almost_equal(all_sampled_data[0], all_sampled_data[1])

AssertionError: 
Arrays are not almost equal to 6 decimals

Mismatched elements: 1000 / 1000 (100%)
Max absolute difference: 8410
Max relative difference: 7715.
 x: array([4046, 1870, 2029,  453,  748, 2145, 2063, 2829, 2115, 7694, 4723,
        360, 1046, 3715, 4035, 1199, 3404, 3096, 7435, 3534, 6567, 8131,
       6790, 5544,  811, 6869, 1803, 5296, 1010, 5972, 7197, 2498, 2215,...
 y: array([3140, 5617, 6480, 3721, 6066, 7721, 3577, 8036, 7712, 7285, 3310,
       6560, 6314, 2742, 4432, 2120, 5043, 8088, 3997, 1256, 2740, 6403,
       2701, 7049, 2862, 6198, 1680, 7936, 2358, 3645, 7396, 5159,  907,...

In [7]:
trainer._load_from_checkpoint("/Users/garylai/Dev/subset-active-learning/results/checkpoints/random_sampling_4000_1500")

Loading model from /Users/garylai/Dev/subset-active-learning/results/checkpoints/random_sampling_4000_1500).
You are resuming training from a checkpoint trained with 4.19.2 of Transformers but your current version is 4.19.1. This is not recommended and could yield to errors or unwanted behaviors.


In [12]:
trainer.train_dataset, trainer.eval_dataset

(Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 2000
 }),
 None)

In [12]:
for i, sampling_size in enumerate(self.config.sampling_sizes):
    n_new_samples = (
        sampling_size if i == 0 else self.config.sampling_sizes[i] - self.config.sampling_sizes[i - 1]
    )
    print(f"Sampling {n_new_samples} new samples")

Sampling 1000 new samples
Sampling 1000 new samples
Sampling 1000 new samples
Sampling 1000 new samples
